In [7]:
import numpy as np
import pandas as pd 

import requests
from bs4 import BeautifulSoup 

In [8]:
result = requests.get("https://www.basketball-reference.com/players/b/bryanko01.html")
print(result.status_code)

src = result.content
#print(src)


soup = BeautifulSoup(src, 'lxml')

#parse page to find all html tags containing rows ('tr')
rows = soup.findAll('tr')
print(rows[:10])


200
[<tr>
<th aria-label="If listed as single number, the year the season ended.★ - Indicates All-Star for league.Only on regular season tables." class="poptip sort_default_asc center" data-stat="season" data-tip="If listed as single number, the year the season ended.&lt;br&gt;★ - Indicates All-Star for league.&lt;br&gt;Only on regular season tables." scope="col">Season</th>
<th aria-label="Age of Player at the start of February 1st of that season." class="poptip sort_default_asc center" data-stat="age" data-tip="Age of Player at the start of February 1st of that season." scope="col">Age</th>
<th aria-label="Team" class="poptip sort_default_asc center" data-stat="team_id" data-tip="Team" scope="col">Tm</th>
<th aria-label="League" class="poptip sort_default_asc center" data-stat="lg_id" data-tip="League" scope="col">Lg</th>
<th aria-label="Position" class="poptip sort_default_asc center" data-stat="pos" data-tip="Position" scope="col">Pos</th>
<th aria-label="Games" class="poptip cente

In [9]:
#extract only the text from each cell in the table

list_rows = []

for row in rows:
	row_td = row.findAll('td')
	str_cells = str(row_td)
	cleantext = BeautifulSoup(str_cells,'lxml').get_text()
	list_rows.append(cleantext)

#print(list_rows)


df = pd.DataFrame(list_rows)

In [10]:
#data cleaning and concatenation

df1 = df[0].str.split(',', expand = True)

df1[0] = df1[0].str.strip('[')
df1[28] = df1[28].str.strip(']')
df1[29] = df1[28] #add arbitrary column for shift

#print(df1.head(10))


col_head = soup.findAll('th')
#print(col_head)

all_header = []
str_cols = str(col_head)
cleantext2 = BeautifulSoup(str_cols, 'lxml').get_text()
all_header.append(cleantext2)

df2 = pd.DataFrame(all_header)


df3 = df2[0].str.split(',', expand = True)


df3[0] = df3[0].str.strip('[')
df3[50] = df3[50].str.strip(']')

df4 = df3.iloc[0]

headers = df3.iloc[:, :30]
#print(headers)

season_info = pd.DataFrame(df4[30:])
#print(season_info)

df5 = df1.shift(periods = 1, axis = 'columns')
#print(df5)


frames = [headers,df5]

df6 = pd.concat(frames)

df6 = df6.rename(columns = df6.iloc[0])

df7 = df6.drop(df6.index[0])
#print(df7)


season_info = season_info.set_index(df7.index)
#print(season_info)


kobe_stats = pd.concat([season_info, df7], axis = 'columns', join_axes = [df7.index])
#print(kobe_stats)

kobe_stats.Season.fillna(season_info[0], inplace = True)


del kobe_stats[0]

print(kobe_stats)

kobe_stats.to_csv('./kobe_stats.csv')

      Season  Age    Tm    Lg  Pos      G     GS     MP     FG    FGA  ...  \
1    1996-97   18   LAL   NBA   SG     71      6   15.5    2.5    5.9  ...   
2    1997-98   19   LAL   NBA   SG     79      1   26.0    4.9   11.6  ...   
3    1998-99   20   LAL   NBA   SG     50     50   37.9    7.2   15.6  ...   
4    1999-00   21   LAL   NBA   SG     66     62   38.2    8.4   17.9  ...   
5    2000-01   22   LAL   NBA   SG     68     68   40.9   10.3   22.2  ...   
6    2001-02   23   LAL   NBA   SG     80     80   38.3    9.4   20.0  ...   
7    2002-03   24   LAL   NBA   SG     82     82   41.5   10.6   23.5  ...   
8    2003-04   25   LAL   NBA   SG     65     64   37.6    7.9   18.1  ...   
9    2004-05   26   LAL   NBA   SG     66     66   40.7    8.7   20.1  ...   
10   2005-06   27   LAL   NBA   SG     80     80   41.0   12.2   27.2  ...   
11   2006-07   28   LAL   NBA   SG     77     77   40.8   10.6   22.8  ...   
12   2007-08   29   LAL   NBA   SG     82     82   38.9    9.5  